In [52]:
# gradient boosting for classification in scikit-learn
import pandas as pd
import numpy as np

from numpy import mean
from numpy import std
from sklearn.datasets import make_classification
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from matplotlib import pyplot
# define dataset
# X, y = make_classification(n_samples=1000, n_features=10, n_informative=5, n_redundant=5, random_state=1)


In [54]:
data = pd.read_csv('data.csv')
data = data.drop(columns=['Date','BTCTOMORROW'])
data.tail(3)

,BTC,ETH,XRP,USDT,BSV,BCH,LTC,EOS,BNB,ADA
670,10242.35,351.11,0.24,1.0,169.58,224.68,47.90,2.76,24.53,0.09
671,10363.14,368.10,0.24,1.0,166.70,227.71,48.72,2.78,24.56,0.10
672,10400.91,374.70,0.24,1.0,167.56,225.54,49.09,2.79,25.45,0.10


In [55]:
# Features construction 
#data['Change BTC (USD)'] = (data['BTC'].shift(-1) - data['BTC'])
#data['GROW'] = np.where(data['Change BTC (USD)'].shift(-1) > data['Change BTC (USD)'], 1, -1)
data['percent_change'] = ((data['BTC'].shift(-1)-data['BTC'])/data['BTC']) * 100
data.dropna(inplace=True)

# X is the input variable
X = data

# Y is the target or output variable
y = np.where((data['BTC'].shift(-1) - data['BTC']) > 0, 1, 0)
data.tail(10)

,BTC,ETH,XRP,USDT,BSV,BCH,LTC,EOS,BNB,ADA,percent_change
662,11970.48,477.05,0.30,1.00,203.67,292.72,62.88,3.48,24.82,0.13,-4.648519
663,11414.03,440.04,0.28,1.00,183.57,264.12,58.10,3.10,24.69,0.12,-10.239416
664,10245.30,385.67,0.25,1.01,152.35,216.06,47.77,2.64,20.62,0.10,2.601290
665,10511.81,388.24,0.26,1.00,160.56,231.87,50.61,3.01,21.06,0.10,-3.255767
666,10169.57,335.26,0.24,1.00,162.10,225.30,47.59,2.93,19.48,0.09,1.089328
667,10280.35,353.36,0.24,1.00,163.50,228.45,48.05,2.91,23.19,0.09,0.867772
668,10369.56,352.67,0.24,1.00,174.15,228.61,48.77,2.85,22.39,0.09,-2.295565
669,10131.52,337.60,0.24,1.00,170.57,222.54,47.38,2.76,23.19,0.09,1.093913
670,10242.35,351.11,0.24,1.00,169.58,224.68,47.90,2.76,24.53,0.09,1.179319
671,10363.14,368.10,0.24,1.00,166.70,227.71,48.72,2.78,24.56,0.10,0.364465


In [56]:
# Total dataset length
dataset_length = data.shape[0]

# Training dataset length
split = int(dataset_length * 0.80)
split

537

In [57]:
# Splitiing the X and y into train and test datasets
X_train, X_test = X[:split], X[split:]
y_train, y_test = y[:split], y[split:]

# Print the size of the train and test dataset
print(X_train.shape, X_test.shape)
print(y_train.shape, y_test.shape)

(537, 11) (135, 11)
(537,) (135,)


In [58]:
# evaluate the model
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
n_scores = cross_val_score(model, X_train, y_train, scoring='accuracy', cv=cv, n_jobs=-1, error_score='raise')
print('Accuracy: %.3f (%.3f)' % (mean(n_scores), std(n_scores)))
# fit the model on the whole dataset
model = GradientBoostingClassifier()
model.fit(X_train, y_train)
# make a single prediction
# row = [[10400.91,374.7,0.24,1,167.56,225.54,49.09,2.79,25.45,0.1,10442.17]]
# yhat = model.predict(row)
# print('Prediction: %d' % yhat[0])

Accuracy: 1.000 (0.000)


GradientBoostingClassifier()

In [59]:
from sklearn.metrics import accuracy_score
y_pred = model.predict(X_test)
print('Correct Prediction (%): ', accuracy_score(y_test, y_pred, normalize=True) * 100.0)

Correct Prediction (%):  98.51851851851852


In [60]:
# classification report metrics
from sklearn.metrics import classification_report
report = classification_report(y_test, model.predict(X_test))
print(report)

              precision    recall  f1-score   support

           0       1.00      0.97      0.98        61
           1       0.97      1.00      0.99        74

    accuracy                           0.99       135
   macro avg       0.99      0.98      0.98       135
weighted avg       0.99      0.99      0.99       135

